In [1]:

from langchain_openai import ChatOpenAI
import os
from openai import api_key, base_url
QWEN_API_KEY = os.getenv("QWEN_API_KEY", "sk-ypjnechwutigvbyglbkguukzsmzkkxfibauydwkbjrypwojd")
llm = ChatOpenAI(model="Qwen/Qwen3-30B-A3B-Instruct-2507", api_key=QWEN_API_KEY, base_url="https://api.siliconflow.cn/v1")


In [ ]:
from langchain_community.utilities import SerpAPIWrapper
SERPAPI_API_KEY = "001bd195e11581af719e2f10bfb5abd7db5f51ffb37f79b2ef7cd43af0b8e7a2"
search = SerpAPIWrapper(serpapi_api_key=SERPAPI_API_KEY)
search.run("刘亦菲最近有什么活动")

"['刘亦菲最近几天在成都工作，参加某商业活动。忙完工作后的她开启了逛街购物的模式，在太古里被偶遇了。 该网友夸赞刘亦菲气质太好了！', '最近上海连办两场大活动，一个是白玉兰奖提名晚宴，另一个是LV的时装秀。刘亦菲因为时间冲突没去LV，但热搜前十有近一半都是她。LV现场来了宋茜、朱一龙这些顶流， ...', '6月20日蚂蚁保非公开活动，菲吧获得15个名额，现进行不同渠道的招募，请大家查看完下述必须符合的条件后，挑选自己适合的渠道进行报名： 1、报名者必须是蚂蚁保用户且“我的 ...', '資生堂| SHISEIDO 全球品牌代言人劉亦菲x 資生堂專訪：極上御藏系列 Global Brand Ambassador Liu Yifei x Shiseido Exclusive Interview：FUTURE SOLUTION ✓ Weibo ...', '... 有杨紫、闫妮这些老戏骨，人家演技确实比偶像派强，刘亦菲还放了白玉兰颁奖礼的鸽子，之前为了去提名晚宴推掉LV活动，结果颁奖那天人影都没见着，这也 ...', '刘亦菲近期参加了多个活动。她出席宝格丽活动，身着长裙，搭配光彩夺目的珠宝，尽显贵气迷人，其美在活动现场十分抢镜，且她在活动中的表现大方得体，无论是舞台上还是舞会环节都 ...', '曾经的刘亦菲凭借“神仙姐姐”这个称号，就可以在圈内获得绝对的位置，在参加相关活动时也能够得到主办方的肯定。 · 直接就把c位给到了她的手上。 · 今年9月3日 ...', '刘亦菲线下见面会来咯！深圳吃的太好了吧。活动项目 ：刘亦菲品牌发布会·线下活动活动时间🏙️：2025年4月17日活动地点⏰：深圳到场嘉宾 ：刘亦菲活动内容 ：签到、 ...', '她的最新动态，是为某奢侈品牌站台，从服装到配饰再到发型都精心挑选搭配，而这次活动还邀请了好莱坞明星安妮·海瑟薇到场，并和刘亦菲亲密互动。', '3月19日，2025上海影视创制大会系列活动——剧耀东方·2025电视剧品质盛典即将举行。活动汇聚《六姊妹》《玫瑰的故事》等多部年度口碑佳作的主创团队，展现 ...']"

In [8]:
from langchain.agents import Tool
searchTool = Tool(
    name="search",
    description="useful for when you need to answer questions about current events",
    func=search.run,
)


In [195]:
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.huggingface import HuggingFaceEmbeddings

embedding_path = "/home/zpc/langchain/blibli/bge-large-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_path)

loader = TextLoader("/home/zpc/langchain/blibli/data/faq_1.txt")
doc = loader.load()
doc

OutOfMemoryError: CUDA out of memory. Tried to allocate 84.00 MiB. GPU 0 has a total capacity of 39.49 GiB of which 8.94 MiB is free. Process 1942885 has 38.62 GiB memory in use. Process 2129480 has 870.00 MiB memory in use. Of the allocated memory 436.93 MiB is allocated by PyTorch, and 17.07 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [105]:
loader = TextLoader("/home/zpc/langchain/blibli/data/faq_2.txt")
doc1 = loader.load()
loader = TextLoader("/home/zpc/langchain/blibli/data/animation.html")
doc2 = loader.load()

In [ ]:

from langchain_core.tools.retriever import create_retriever_tool


vectorStoreDB = FAISS.from_documents([doc[0], doc1[0], doc2[0]],embedding=embeddings)
retriver = vectorStoreDB.as_retriever(
    search_type="mmr",
    search_kwargs={"k":1}
)
retriver_tool = create_retriever_tool(
    retriver,
    name="retriver",
    "华为商城帮助中心文档搜索"
)

tools = [retriver_tool, searchTool]

NameError: name 'embeddings' is not defined

In [ ]:
retriver_tool.invoke("众测活动")

In [193]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
print(prompt)


input_variables=['agent_scratchpad', 'input'] optional_variables=['chat_history'] input_types={'chat_history': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')]

In [ ]:
from langchain.agents import create_openai_functions_agent
agent = create_openai_functions_agent(llm, tools, verbose=True)
agent

In [ ]:
from langchain.agents import AgentExecutor
agent_exexutor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_exexutor